In [40]:
MODEL = "gpt-4o-2024-08-06"
#MODEL = "gpt-4o-mini"
#MODEL = "gpt-3.5-turbo"
prompt = [{"role": "user", "content": ""}]
loops = 100
load_file = "dataset"
save_file = "dataset"

import os
from openai import OpenAI
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()
df_path = os.path.join(os.getcwd())

In [42]:
# Load completions from json, if continuing with a file generated in the past

try:
    with open(load_file, "r") as f:
        completions = pd.read_json(f, lines=True)

# If not, create a new completions database

except FileNotFoundError:
    dataset = pd.DataFrame([],columns=['Response', 'Eval', 'Data', 'Instruction'])

In [9]:
for i in range(loops):
    completion = client.chat.completions.create(
        messages = prompt,
        model = MODEL,
        )
    response = completion.choices[0].message.content

    data = {'Response': response, 'Eval': "", 'Data': {}, 'Instruction': prompt}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict()}
    dataset = pd.concat([dataset, data],ignore_index=True)
    dataset.to_json(os.path.join(df_path, file_name), lines=True, orient="records")

In [44]:
completions

,Response,Eval,Data,Instruction
0,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhCemTLIU9rK...,"[{'role': 'user', 'content': ''}]"
1,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhDUWFpckVaq...,"[{'role': 'user', 'content': ''}]"
2,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhDwjYX970LS...,"[{'role': 'user', 'content': ''}]"
3,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhEM6HEaNeZW...,"[{'role': 'user', 'content': ''}]"
4,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhFU3uvCpw0i...,"[{'role': 'user', 'content': ''}]"
...,...,...,...,...
95,It seems you've submitted an empty message. Ho...,,{'Completion': {'id': 'chatcmpl-9wPiVTdzVMfOAG...,"[{'role': 'user', 'content': ''}]"
96,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiWXaGj5uqvW...,"[{'role': 'user', 'content': ''}]"
97,It seems like your message got cut off or you ...,,{'Completion': {'id': 'chatcmpl-9wPiWzXUC8Zwer...,"[{'role': 'user', 'content': ''}]"
98,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiXkuFd5J53C...,"[{'role': 'user', 'content': ''}]"


In [34]:
dataset.sort_values("Response")

,Response,Eval,Data,Instruction
0,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhCemTLIU9rK...,"[{'role': 'user', 'content': ''}]"
65,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiBnr6UffbYo...,"[{'role': 'user', 'content': ''}]"
64,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiAFvZRGSZzm...,"[{'role': 'user', 'content': ''}]"
63,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiAkMOYLPLup...,"[{'role': 'user', 'content': ''}]"
62,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPi9ZkCQm55MR...,"[{'role': 'user', 'content': ''}]"
...,...,...,...,...
88,It seems like your message didn't come through...,,{'Completion': {'id': 'chatcmpl-9wPiQiC4FDSETE...,"[{'role': 'user', 'content': ''}]"
52,It seems like your message got cut off or wasn...,,{'Completion': {'id': 'chatcmpl-9wPi1NiGVlgrYj...,"[{'role': 'user', 'content': ''}]"
97,It seems like your message got cut off or you ...,,{'Completion': {'id': 'chatcmpl-9wPiWzXUC8Zwer...,"[{'role': 'user', 'content': ''}]"
53,It seems like your message was empty. Could yo...,,{'Completion': {'id': 'chatcmpl-9wPi1bwEROlK5a...,"[{'role': 'user', 'content': ''}]"


In [33]:
sorted

,Response,Eval,Data,Instruction
0,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhCemTLIU9rK...,"[{'role': 'user', 'content': ''}]"
65,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiBnr6UffbYo...,"[{'role': 'user', 'content': ''}]"
64,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiAFvZRGSZzm...,"[{'role': 'user', 'content': ''}]"
63,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiAkMOYLPLup...,"[{'role': 'user', 'content': ''}]"
62,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPi9ZkCQm55MR...,"[{'role': 'user', 'content': ''}]"
...,...,...,...,...
88,It seems like your message didn't come through...,,{'Completion': {'id': 'chatcmpl-9wPiQiC4FDSETE...,"[{'role': 'user', 'content': ''}]"
52,It seems like your message got cut off or wasn...,,{'Completion': {'id': 'chatcmpl-9wPi1NiGVlgrYj...,"[{'role': 'user', 'content': ''}]"
97,It seems like your message got cut off or you ...,,{'Completion': {'id': 'chatcmpl-9wPiWzXUC8Zwer...,"[{'role': 'user', 'content': ''}]"
53,It seems like your message was empty. Could yo...,,{'Completion': {'id': 'chatcmpl-9wPi1bwEROlK5a...,"[{'role': 'user', 'content': ''}]"
